<a href="https://colab.research.google.com/github/giuliocapecchi/IR_project/blob/main/IR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch matplotlib nltk tqdm gdown

# 1. Download the collection

Con questo modulo si possono scaricare files, quindi ho scaricato la collection e l'ho butttata sul mio drive (ci vogliono circa 30s/1 minuto di tempo)

In [2]:
import gdown

url = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
gdown.download(url, 'collection.tsv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn
From (redirected): https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn&confirm=t&uuid=d389e6ba-7a60-43ba-8444-500de75e0630
To: /content/collection.tsv
100%|██████████| 3.06G/3.06G [00:40<00:00, 75.2MB/s]


'collection.tsv'

In [3]:
"""
read 'collection.tsv' file and prepare it for data manipulation
the file is organized in the following way:
<pid>\t<text>\n
where <pid> is the passage id and <text> is the passage text
"""
import pandas as pd

df = pd.read_csv('collection.tsv', sep='\t', header=None)

In [4]:
# let's not truncate Pandas output too much
pd.set_option('display.max_colwidth', 50) # mettici 150
df.columns = ['doc_id', 'text']
print(df.head(2)) # returns the first N rows

   doc_id                                               text
0       0  The presence of communication amid scientific ...
1       1  The Manhattan Project and its atomic bomb help...


In [5]:
import re
import string
import nltk

nltk.download("stopwords", quiet=True)

def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand
    s = s.replace("&", " and ")
    # special chars
    s = s.translate(dict([(ord(x), ord(y)) for x, y in zip("‘’´“”–-", "'''\"\"--")]))
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # remove dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = s.strip()
    while "  " in s:
        s = s.replace("  ", " ")
    # tokeniser
    s = s.split()
    # stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    s = [t for t in s if t not in stopwords]
    # stemming
    stemmer = nltk.stem.PorterStemmer().stem
    s = [stemmer(t) for t in s]
    return s

In [6]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [39]:
def VBEncode(n):
    byte = [] # We will store the bytes in a list
    while True:
        # n % 128 gives us the 7 least significant bits
        byte.append(n % 128) # Append 7 bits with most significant bit (control bit) set to 0
        if n < 128:
            break
        n //= 128 # Shift the number to the right by 7 bits

    byte[0] += 128 # Set the most significant bit (control bit) set to 1
    return byte[::-1] # We provide the resulting list in reverse order


def VBEncodeList(n_list):
    b = []
    for n in n_list:
        b.extend(VBEncode(n)) # We extend the list with the bytes of the number
    return b


def VBDecode(byte_list):
    n_list = []
    n = 0
    for b in byte_list:
        if b < 128:
            n = 128 * n + b
        else:
            n = 128 * n + b - 128
            n_list.append(n)
            n = 0
    return n_list


print("Initial list: [5, 824] -> Encoded list:", VBEncodeList([5, 824]))
print("Encoded ist: [133, 6, 184] -> Decoded list:", VBDecode([133, 6, 184]))

Initial list: [5, 824] -> Encoded list: [133, 6, 184]
Encoded ist: [133, 6, 184] -> Decoded list: [5, 824]


In [41]:
from collections import Counter
from tqdm.auto import tqdm

@profile
def build_index(dataset):
    lexicon = {}
    doc_index = []
    inv_d, inv_f = {}, {}
    termid = 0

    num_docs = 0
    total_dl = 0
    total_toks = 0
    for docid, doc in tqdm(enumerate(dataset.docs_iter()), desc='Indexing', total=dataset.docs_count()):
        tokens = preprocess(doc.text)
        #print(tokens)
        token_tf = Counter(tokens)
        for token, tf in token_tf.items():
            if token not in lexicon:
                lexicon[token] = [termid, 0, 0]
                inv_d[termid], inv_f[termid] =  [], []
                termid += 1
            token_id = lexicon[token][0] # prendo il termid
            inv_d[token_id].append(docid) # aggiungo il docid alla lista dei docid in cui compare il termine
            inv_f[token_id].append(tf) # aggiungo il tf alla lista dei tf in cui compare il termine
            lexicon[token][1] += 1 # incremento il df
            lexicon[token][2] += tf # tf è quanto compare il termine nel documento
        doclen = len(tokens)
        doc_index.append((str(doc.doc_id), doclen))
        total_dl += doclen
        num_docs += 1


    stats = {
        'num_docs': 1 + docid, # docid parte da 0
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [42]:
"""
This class that takes the dataframe we created before with columns 'docno' and 'text', and creates a list of namedtuples
"""
from collections import namedtuple


class MSMarcoDataset:
    def __init__(self, df):
        self.docs = [Document(row.doc_id, row.text) for row in df.itertuples()]

    def docs_iter(self):
        return iter(self.docs)

    def docs_count(self):
        return len(self.docs)


Document = namedtuple('Document', ['doc_id', 'text']) # must define what a document is

In [43]:
# Test the MSMarcoDataset class by passing Document(1, "school"), Document(2, "example."), Document(3, "house.")

test_docs = [Document(1, "A school"), Document(2, "Another example."), Document(3, "This is a house.")]
test_dataset = MSMarcoDataset(pd.DataFrame(test_docs, columns=['doc_id', 'text']))

for doc in test_dataset.docs_iter():
    print(doc)

lex, inv, doc, stats = build_index(test_dataset)


Document(doc_id=1, text='A school')
Document(doc_id=2, text='Another example.')
Document(doc_id=3, text='This is a house.')


Indexing:   0%|          | 0/3 [00:00<?, ?it/s]

build_index (38.953 ms)


In [44]:
print(lex)
print(inv)
print(doc)
print(stats)

{'school': [0, 1, 1], 'anoth': [1, 1, 1], 'exampl': [2, 1, 1], 'hous': [3, 1, 1]}
{'docids': {0: [128], 1: [129], 2: [129], 3: [130]}, 'freqs': {0: [129], 1: [129], 2: [129], 3: [129]}}
[('1', 1), ('2', 2), ('3', 1)]
{'num_docs': 3, 'num_terms': 4, 'num_tokens': 4}


In [45]:
import math

class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stat = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        return InvertedIndex.PostingListIterator(self.inv['docids'][termid], self.inv['freqs'][termid], self.doc)

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]

In [46]:
inv_index = InvertedIndex(lex, inv, doc, stats)

TopQueue class, which stores the top  K  (score, docid) tuples, using an heap

In [47]:
import heapq

class TopQueue:
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

TAAT

In [48]:
from collections import defaultdict

def taat(postings, k=10):
    A = defaultdict(float)
    for posting in postings:
        current_docid = posting.docid()
        while current_docid != math.inf:
            A[current_docid] += posting.score()
            posting.next()
            current_docid = posting.docid()
    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return taat(postings)

DAAT

In [49]:
import math

def min_docid(postings):
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat(postings, k=10):
    top = TopQueue(k)
    current_docid = min_docid(postings)
    while current_docid != math.inf:
        score = 0
        next_docid = math.inf
        for posting in postings:
            if posting.docid() == current_docid:
                score += posting.score()
                posting.next()
            if not posting.is_end_list():
                next_docid = posting.docid()
        top.insert(current_docid, score)
        current_docid = next_docid
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return daat(postings)

In [50]:
url = 'https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz'
gdown.download(url, 'queries.tar.gz', quiet=False)
!tar -xzf queries.tar.gz


Downloading...
From: https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz
To: /content/queries.tar.gz
100%|██████████| 18.9M/18.9M [00:01<00:00, 15.0MB/s]


In [51]:
import pandas as pd


queries = pd.read_csv('queries.eval.tsv', sep='\t', header=None)

In [52]:
class MSMarcoQueries:
    def __init__(self, df):
        self.queries = [Query(row.query_id, row.text) for row in df.itertuples()]

    def queries_iter(self):
        return iter(self.queries)

    def queries_count(self):
        return len(self.queries)


Query = namedtuple('Query', ['query_id', 'text'])

In [53]:
queries.columns = ['query_id', 'text']
queries_dataset = MSMarcoQueries(queries)

In [54]:
print(len(list(queries_dataset.queries_iter())))

101092


In [55]:
@profile
def query_processing(queries_iter, fn):
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res = fn(postings)

In [56]:
query_processing(queries_dataset.queries_iter(), taat)

IndexError: list index out of range

In [ ]:
query_processing(queries_dataset.queries_iter(), daat)

Now build up the index for the MSMARCO collection

In [ ]:
# create a df with the first 10 rows
#df_cut = df.head(10) # TODO : REMOVE THIS

dataset = MSMarcoDataset(df)

The index is built only if a pickled version isn't already present

In [ ]:
import pickle

try: # try to open the pickled files, else build the index
    with open('./lex.pkl', 'rb') as f:
        lex = pickle.load(f)
    with open('./inv.pkl', 'rb') as f:
        inv = pickle.load(f)
    with open('./doc.pkl', 'rb') as f:
        doc = pickle.load(f)
    with open('./stats.pkl', 'rb') as f:
        stats = pickle.load(f)
    print("Index loaded from pickles")

except:
    lex, inv, doc, stats = build_index(dataset)

    # pickle lex, inv, doc, stats
    with open('./lex.pkl', 'wb') as f:
        pickle.dump(lex, f)

    with open('./inv.pkl','wb') as f:
        pickle.dump(inv, f)

    with open('./doc.pkl', 'wb') as f:
        pickle.dump(doc, f)

    with open('./stats.pkl', 'wb') as f:
        pickle.dump(stats, f)

    print("Index built and pickled")

In [ ]:
print(lex)
print(inv)
print(doc)
print(stats)

In [ ]:
inv_index = InvertedIndex(lex, inv, doc, stats)

In [ ]:
query_processing(queries_dataset.queries_iter(), taat)

In [ ]:

query_processing(queries_dataset.queries_iter(), daat)